高斯滤波

In [ ]:
import cv2  as cv
import numpy as np


img = cv.imread(PictureAddress)                            # 高斯滤波,主要用于图像平滑和降噪处理
gaussian = cv.GaussianBlur(img, (5, 5), 2)    # (5,5)：5x5高斯核（必须为奇数），核尺寸越大模糊效果越强，会损失图像高频信息（边缘细节），2：X方向标准差（σ），值越大模糊越强
#gaussian = cv.GaussianBlur(img, (5, 5), 0)                # σ=0时自动根据核尺寸计算σ值
 
cv.imshow("input", img)                                    # 显示原始图像
cv.imshow('output',gaussian)                               # 显示高斯模糊后的图像

cv.waitKey(0)                                              # 等待按键
cv.destroyAllWindows()                                     # 关闭所有窗口

均值滤波

In [ ]:
import cv2 as cv

img = cv.imread(PictureAddress)                 # 读取图像文件
# 均值滤波，该操作可有效抑制高斯噪声和随机噪声，但会导致边缘模糊，对椒盐噪声效果较差，此时建议改用中值滤波（cv2.medianBlur()），均值滤波时高频细节（如边缘）会被平滑，需权衡去噪与特征保留
blur = cv.blur(img, (3, 3))                     # 核尺寸过大会导致过度模糊，建议根据噪声程度选择3×3或5×5
dst= blur
cv.imshow('output',dst)
cv.imshow("input", img)

# 补充
#def blur_demo(image):                          # 定义一个均值滤波函数
    #dst = cv.blur(image, (3, 3))
    #cv.imshow('blur',dst)
#src= cv.imread("D:/images/nezha.png")
#cv.imshow("input", src)
#blur_demo(src)

cv.waitKey(0)                                   # 等待按键
cv.destroyAllWindows()                          # 关闭所有窗口

双边滤波

In [ ]:
import cv2 as cv
import numpy as np

src = cv.imread(PictureAddress)                                    # 读取图像文件
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                        # 创建可自动调整大小的窗口
cv.imshow("input", src)                                            # 显示图像
# cv.bilateralFilter（）即双边滤波，主要用于图像平滑处理的同时保留边缘细节   9：滤波核直径（必须是正奇数）
dst = cv.bilateralFilter(src, 9, 100,10)   # 100：颜色空间滤波参数（值越大，颜色相近的像素影响越大） 10：坐标空间滤波参数（值越大，距离越远的像素影响越大）
output = dst                                                       # 与高斯滤波对比,双边滤波能保留边缘但计算更耗时,高斯滤波会模糊所有高频信息
cv.imshow('output',output)

cv.waitKey(0)                                                      # 等待按键
cv.destroyAllWindows()                                             # 关闭所有窗口

中值滤波

In [ ]:
import cv2 as cv
from numpy.ma.extras import dstack

img = cv.imread(PictureAddress)                    # 读取图像文件
# cv.medianBlur是中值滤波函数，用于图像去噪       5：滤波核的尺寸（必须是大于1的奇数）
median = cv.medianBlur(img, 5)                     # 中值滤波,用窗口内像素值的中位数替代中心像素值，有效消除孤立噪声点
dst=median                                         # 相比均值滤波（cv2.blur），能更好保留边缘且对椒盐噪声效果显著
cv.imshow("input",img)                             # 计算耗时约为均值滤波的5倍,大核尺寸（如7×7）会显著增加处理时间
cv.imshow('output',dst)

# 补充
#def median_blur_demo(image):                      # 定义一个中值滤波的函数
    #dst=cv.medianBlur(image,5) 
    #cv.imshow('output',dst)
#image= cv.imread("D:/images/nezha.png")
#cv.imshow("input",img)
#median_blur_demo(image)


cv.waitKey(0)                                      # 等待按键
cv.destroyAllWindows()                             # 关闭所有窗口


边缘滤波

In [ ]:
import cv2 as cv
import numpy as np


src = cv.imread(PictureAddress)                           # 读取图像文件
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)               # 创建可自动调整大小的窗口
cv.imshow("input", src)                                   # 显示图像

h, w = src.shape[:2]                                      # 获取图像宽高
#cv.edgePreservingFilter()是用于实现边缘保留滤波的核心函数，主要用于图像平滑处理时保护边缘结构不被模糊
dst = cv.edgePreservingFilter(src, sigma_s=100, sigma_r=0.4, flags=cv.RECURS_FILTER)  # sigma_s控制空间域平滑，sigma_r控制颜色域平滑，采用递归滤波方式(flags=cv.RECURS_FILTER)
result = np.zeros([h, w*2, 3], dtype=src.dtype)           # 创建左右并排的对比画布
result[0:h,0:w,:] = src                                   # 左侧显示原图
result[0:h,w:2*w,:] = dst                                 # 右侧显示处理结果

cv.imshow("result", result)                               # 显示处理后的图像结果
cv.imwrite("D:/images/result.png", result)                # 将结果图像保存到指定路径

cv.waitKey(0)                                             # 等待按键
cv.destroyAllWindows()                                    # 关闭所有窗口

均值迁移模糊

In [ ]:
import cv2 as cv
import numpy as np


# 均值迁移模糊，实现边缘保留的平滑效果，常用于去除噪声同时保持边缘清晰
# 对比双边滤波，均值迁移滤波通过迭代收敛更适用于复杂色彩分布的场景，但计算量较大，双边滤波则直接基于空间和色彩权重计算，实时性更好
src = cv.imread(PictureAddress)                           # 读取图像文件
cv.imshow('input',src)                                    # 显示输入图像
# 均值漂移滤波（Mean Shift Filtering），主要用于图像分割和边缘保留平滑处理   参数15表示空间窗口半径，30表示颜色窗口半径   终止条件设置为最大迭代5次或误差小于1
dst = cv.pyrMeanShiftFiltering(src, 15, 30, termcrit=(cv.TERM_CRITERIA_MAX_ITER+cv.TERM_CRITERIA_EPS, 5, 1))
cv.imshow('output',dst)                                   # 显示输出图像

cv.waitKey(0)                                             # 等待按键
cv.destroyAllWindows()                                    # 关闭所有窗口

sobel算子

In [ ]:
import cv2 as cv
import numpy as np


image= cv.imread(PictureAddress)                  # 读取图像文件
src= cv.cvtColor(image,cv.COLOR_BGR2GRAY)         # BGR转换为灰度图像
# 使用Sobel算子计算图像的水平和垂直梯度
h, w = src.shape[:2]                              # 获取图像高宽
x_grad = cv.Sobel(src, cv.CV_32F, 1, 0)           #（1，0）计算水平方向（x轴）梯度（检测垂直边缘）
y_grad = cv.Sobel(src, cv.CV_32F, 0, 1)           #（0, 1‌）计算垂直方向（y轴）梯度（检测水平边缘）

# 可视化梯度（将Sobel梯度计算结果(x_grad/y_grad)转换为8位无符号整型（unit.8））
x_grad = cv.convertScaleAbs(x_grad)               # convertScaleAbs()自动完成绝对值转换和数值缩放
y_grad = cv.convertScaleAbs(y_grad)
cv.imshow("x_grad", x_grad)                       # 显示水平方向梯度图像
cv.imshow("y_grad", y_grad)                       # 显示垂直方向梯度图像

# 梯度幅值合成（cv.add）
dst = cv.add(x_grad, y_grad, dtype=cv.CV_16S)     # 将水平梯度x_grad和垂直梯度y_grad按元素相加，生成综合梯度幅值矩阵, 指定输出为16位有符号整数

# 绝对值转换（cv.convertScaleAbs）
dst = cv.convertScaleAbs(dst)                     # 将合成梯度转换为8位无符号整型（uint8）
cv.imshow('output',dst)                           # 显示梯度合成图像

cv.waitKey(0)                                     # 等待按键
cv.destroyAllWindows()                            # 关闭所有窗口

拉普拉斯算子

In [ ]:
import cv2 as cv
import numpy as np


image = cv.imread(PictureAddress)                      # 读取图像文件
cv.imshow('input',image)                               # 显示图像
#高斯模糊预处理（GaussianBlur函数对输入图像进行平滑处理）
src = cv.GaussianBlur(image, (0, 0), 1)   # (0,0)表示自动计算核尺寸，设置标准差σ=1控制模糊程度（σ值越大模糊效果越明显）
cv.imshow('gaussian',src)                              # 显示高斯模糊后的图像

#拉普拉斯边缘检测,对模糊后图像应用cv.Laplacian二阶微分算子
dst = cv.Laplacian(src, cv.CV_32F, ksize=3, delta=127) # 表示使用3×3的Sobel核对输入图像src进行二阶微分运算,指定输出32位浮点型数据（保留负梯度值）,delta=127将所有结果偏移127（便于可视化处理）

#图像数据转换
dst = cv.convertScaleAbs(dst)                          # 对输入数组执行线性变换并取绝对值后转换为8位无符号整型
cv.imshow('output',dst)                                # 显示图像

cv.waitKey(0)                                          # 等待按键
cv.destroyAllWindows()                                 # 关

图像积分图算法

In [ ]:
import cv2 as cv
import numpy as np

# 积分图中矩形区域像素值的快速求和计算（核心原理是通过四个角点的积分图值进行加减运算，快速得到任意矩形区域的像素和）
def get_block_sum(ii, x1, y1, x2, y2, index):
    tl = ii[y1, x1][index]                                       # (y1,x1)：矩形区域左上角坐标，y在前符合opencv行列约定
    tr = ii[y2, x1][index]
    bl = ii[y1, x2][index]
    br = ii[y2, x2][index]                                       # (y2,x2)：矩形区域右下角坐标
    s = (br - bl - tr + tl)                                      # 公式 s = br - bl - tr + tl 计算矩形区域和
    return s

# 基于积分图(ii)（Integral Image）的快速均值模糊
def blur_demo(image, ii):
    h, w, dims = image.shape                                     # 获取图像的基本维度信息（高度、宽度和通道数）
    result = np.zeros(image.shape, image.dtype)                  # 创建一个与输入图像尺寸和数据类型完全相同的全零矩阵，常用于图像处理中存储计算结果
    ksize = 15                                                   # 设定卷积核大小为15x15像素
    radius = ksize // 2                                          # 计算核半径（整除运算得7）

    # 该循环配合内层的列循环(col)共同完成对整个图像的遍历，每个(row,col)位置对应一个处理中心点(cy,cx)
    for row in range(0, h + radius, 1):                          # 从0开始到图像高度h加上模糊半径radius，步长为1表示逐行处理
        # 边界条件判断，确保卷积窗口坐标不超出图像范围
        y2 = h if (row + 1)> h else (row + 1)                    # 判断(row + 1)是否超出图像高度h，若超出则取h作为边界值，否则取(row + 1)
        y1 = 0 if (row - ksize) < 0 else (row - ksize)           # 判断(row - ksize)是否小于0（上边界），若越界则取0，否则取(row - ksize)，确保卷积核访问不会越出图像上边界
        for col in range(0, w + radius, 1):                      # range(0, w + radius, 1)表示从0开始到图像宽度w加上模糊半径radius，步长为1实现逐列移动
            x2 = w if (col + 1)>w else (col + 1)                 # 判断(col + 1)是否超出图像宽度w，若超出则取w作为右边界，否则取(col + 1)
            x1 = 0 if (col - ksize) < 0 else (col - ksize)       # 判断(col - ksize)是否小于0（左边界），若越界则取0，否则取(col - ksize)
            # 通过边界检查确保卷积中心点不越界
            cx = 0 if (col - radius) < 0 else (col - radius)
            cy = 0 if (row - radius) < 0 else (row - radius)
            num = (x2 - x1)*(y2 - y1)                            # 通过(x2-x1)*(y2-y1)计算当前卷积窗口的实际像素面积，用于后续归一化处理（如均值模糊）
            #通过range(0,3,1)依次处理BGR三通道，每个通道独立进行卷积运算，通过get_block_sum获取每个通道的矩形区域像素和
            for i in range(0, 3, 1):
                s = get_block_sum(ii, x1, y1, x2, y2, i)         # (x1,y1)-(x2,y2)：矩形区域坐标
                # 将区域和除以num（区域像素数）得到均值，(cy,cx)：结果存储位置
                result[cy, cx][i] = s // num                     # s // num实现窗口内像素值的均值计算，计算结果写入result[cy, cx][i]对应通道

    cv.imshow("integral fast blur", result)                      # 在窗口中显示经过积分图快速模糊处理的结果图像
    cv.imwrite("D:/images/result.png", result)                   # 将处理结果保存到指定路径

src = cv.imread(PictureAddress)                                  # 读取图像文件
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                      # 创建可自动调整大小的窗口
cv.imshow("input", src)                                          # 显示图像

#使用OpenCV的integral()函数计算图像的积分图（和表）
sum_table = cv.integral(src, sdepth=cv.CV_32S)                   # 其中src是输入图像，sdepth=cv.CV_32S指定输出积分图的数据类型为32位有符号整数
blur_demo(src, sum_table)                                        # 利用积分图实现快速区域模糊

cv.waitKey(0)                                                    # 等待按键
cv.destroyAllWindows()                                           # 关闭所有窗口